In [1]:
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# !pip install tableone
from tableone import TableOne
# !pip install psycopg2-binary
import psycopg2

In [2]:
# create a database connection
sqluser = 'asem'
dbname = 'mimiciv'
hostname = 'localhost'
password='qwerasdf'
hosp_schema_name = 'mimiciv_hosp'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser, host=hostname, password=password)

In [3]:
query = \
"""
SELECT
      pat.subject_id
    , adm.hadm_id
    , DENSE_RANK() OVER hadm_window AS hosp_stay_num
    , CASE
        WHEN FIRST_VALUE(adm.hadm_id) OVER hadm_window = adm.hadm_id THEN 1
        ELSE 0
      END AS pat_count
    , pat.anchor_age + (EXTRACT(YEAR FROM adm.admittime) - pat.anchor_year) AS age
    , pat.gender
    , adm.insurance
    , mimiciv_derived.DATETIME_DIFF(adm.dischtime, adm.admittime, 'HOUR') / 24 AS hosp_los
    , pat.dod
    , mimiciv_derived.DATETIME_DIFF(pat.dod, CAST(adm.dischtime AS DATE), 'DAY') AS days_to_death
    -- mortality flags
    , CASE WHEN mimiciv_derived.DATETIME_DIFF(pat.dod, CAST(adm.dischtime AS DATE), 'DAY') = 0 THEN 1 ELSE 0 END AS hospital_mortality
FROM mimiciv_hosp.patients pat
INNER JOIN mimiciv_hosp.admissions adm
    ON pat.subject_id = adm.subject_id
WINDOW hadm_window AS (PARTITION BY pat.subject_id ORDER BY adm.admittime)
"""

hosp = pd.read_sql_query(query,con)

In [4]:
# Admission IDs with Length of Stay at least 12 hours.
hadm_id_los_geq_12h = hosp[hosp.hosp_los > 0.5].hadm_id
hadm_id_los_geq_12h

0         22595853
1         22841357
2         29079034
3         25742920
5         23052089
            ...   
431226    29734428
431227    25744818
431228    26071774
431229    21033226
431230    23865745
Name: hadm_id, Length: 392689, dtype: int64

In [5]:
adm_df = pd.read_sql_query("""
SELECT subject_id, hadm_id, admittime, dischtime FROM mimiciv_hosp.admissions
""", con)
adm_df

,subject_id,hadm_id,admittime,dischtime
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00
...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00


In [6]:

# add 1 year mortality
hosp['one_year_mortality'] = hosp['days_to_death'].notnull().astype(int)

# create a dataframe with the days to death for only the last ICU stay
last_dod = hosp.groupby('subject_id')[['hosp_stay_num']].max().reset_index()
last_dod = last_dod.merge(hosp[['subject_id', 'hosp_stay_num', 'days_to_death']], on=['subject_id', 'hosp_stay_num'], how='inner')
last_dod.rename(columns={'days_to_death': 'days_to_death_last_stay_id'}, inplace=True)

hosp = hosp.merge(last_dod, how='left', on=['subject_id', 'hosp_stay_num'])
del last_dod
hosp.sort_values(['subject_id', 'hosp_stay_num'], inplace=True)

# fix some data type issues
int_cols = hosp.dtypes.values=="Int64"
hosp.loc[:, int_cols] = hosp.loc[:, int_cols].astype(float)
hosp.loc[:, int_cols] = hosp.loc[:, int_cols].astype(int, errors="ignore")


In [7]:
query = \
"""
SELECT
      pat.subject_id
    , adm.hadm_id
    , icu.stay_id
    , ROW_NUMBER() OVER (PARTITION BY pat.subject_id ORDER BY icu.intime) AS icu_stay_num
    , DENSE_RANK() OVER (PARTITION BY pat.subject_id ORDER BY adm.admittime) AS hosp_stay_num
    , CASE
        WHEN FIRST_VALUE(icu.stay_id) OVER icustay_window = icu.stay_id THEN 1
        ELSE 0
      END AS pat_count
    , pat.anchor_age + (EXTRACT(YEAR FROM icu.intime) - pat.anchor_year) AS age
    , pat.gender
    , adm.insurance
    , icu.first_careunit
    , icu.los AS icu_los
    , mimiciv_derived.DATETIME_DIFF(adm.dischtime, adm.admittime, 'HOUR') / 24 AS hosp_los
    , pat.dod
    , mimiciv_derived.DATETIME_DIFF(pat.dod, CAST(adm.dischtime AS DATE), 'DAY') AS days_to_death
    -- mortality flags
    , CASE WHEN mimiciv_derived.DATETIME_DIFF(pat.dod, CAST(adm.dischtime AS DATE), 'DAY') = 0 THEN 1 ELSE 0 END AS hospital_mortality
    , CASE WHEN mimiciv_derived.DATETIME_DIFF(pat.dod, CAST(icu.outtime AS DATE), 'DAY') = 0 THEN 1 ELSE 0 END AS icu_mortality
FROM mimiciv_hosp.patients pat
INNER JOIN mimiciv_hosp.admissions adm
    ON pat.subject_id = adm.subject_id
INNER JOIN mimiciv_icu.icustays icu
    ON adm.hadm_id = icu.hadm_id
WINDOW hadm_window AS (PARTITION BY pat.subject_id ORDER BY adm.admittime)
     , icustay_window AS (PARTITION BY pat.subject_id ORDER BY icu.intime)
"""

data = pd.read_sql_query(query,con)

In [8]:
# add 1 year mortality
data['one_year_mortality'] = data['days_to_death'].notnull().astype(int)

# create a dataframe with the days to death for only the last ICU stay
last_dod = data.groupby('subject_id')[['icu_stay_num']].max().reset_index()
last_dod = last_dod.merge(data[['subject_id', 'icu_stay_num', 'days_to_death']], on=['subject_id', 'icu_stay_num'], how='inner')
last_dod.rename(columns={'days_to_death': 'days_to_death_last_stay_id'}, inplace=True)

data = data.merge(last_dod, how='left', on=['subject_id', 'icu_stay_num'])
del last_dod
data.sort_values(['subject_id', 'icu_stay_num'], inplace=True)

# add a grouping variable for table one so we can have hospital mortality as a group and a row
data['hosp_mort'] = data['hospital_mortality']

# fix some data type issues
int_cols = data.dtypes.values=="Int64"
data.loc[:, int_cols] = data.loc[:, int_cols].astype(float)
data.loc[:, int_cols] = data.loc[:, int_cols].astype(int, errors="ignore")

In [9]:


columns = [
    "pat_count",
    # , "hadm_count",
    "age", "gender", "insurance",
    # "first_careunit",
    # "icu_los",
    "hosp_los",
    # 'icu_mortality',
    'hospital_mortality',
    "one_year_mortality",
    # "days_to_death_last_stay_id"
]

categorical = [
    "pat_count",
    # "hadm_count",
    "gender", "insurance",
    # "first_careunit",
    # mortality flags
    # 'icu_mortality',
    'hospital_mortality',
    'one_year_mortality',
]

order = {
    "pat_count": [1, 0],
    "hadm_count": [1, 0],
    "gender": ["F", "M"],
    # "icu_mortality": [1, 0],
    "hospital_mortality": [1, 0],
    "one_year_mortality": [1, 0],
}

limit = {
    "pat_count": 1, "hadm_count": 1,
    "gender": 1,
    # "icu_mortality": 1,
    "hospital_mortality": 1,
    "one_year_mortality": 1,
}

rename = {
    "pat_count": "Distinct patients", "hadm_count": "Distinct hospitalizations",
    "age": "Age", "gender": "Administrative Gender", "insurance": "Insurance",
    "first_careunit": "First ICU stay, unit type",
    "icu_los": "ICU length of stay", "hosp_los": "Hospital length of stay",
    "icu_mortality": "In-ICU mortality",
    "hospital_mortality": "In-hospital mortality",
    "one_year_mortality": "One year mortality",
    # "days_to_death_last_stay_id": "Time to death (days)",
}

print('ICU demographics')
icu_table = TableOne(data, columns=columns, categorical=categorical, order=order, limit=limit, rename=rename)
display(icu_table)
print('Hospital demographics')
hosp_table = TableOne(hosp, columns=columns, categorical=categorical, order=order, limit=limit, rename=rename)
display(hosp_table)



ICU demographics


/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/tableone/tableone.py:1433: UserWarning: Order variable not found: hadm_count
  warnings.warn("Order variable not found: {}".format(k))


Missing       Overall
n                                                           73181
Distinct patients, n (%)           1              0  50920 (69.6)
Age, mean (SD)                                    0   64.7 (16.9)
Administrative Gender, n (%)       F              0  32363 (44.2)
Insurance, n (%)                   Medicaid       0    5528 (7.6)
                                   Medicare          33091 (45.2)
                                   Other             34562 (47.2)
Hospital length of stay, mean (SD)                0   11.0 (13.3)
In-hospital mortality, n (%)       1              0   8511 (11.6)
One year mortality, n (%)          1              0  28274 (38.6)

Hospital demographics


/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/tableone/tableone.py:1433: UserWarning: Order variable not found: hadm_count
  warnings.warn("Order variable not found: {}".format(k))


Missing        Overall
n                                                           431231
Distinct patients, n (%)           1              0  180733 (41.9)
Age, mean (SD)                                    0    58.8 (19.2)
Administrative Gender, n (%)       F              0  224990 (52.2)
Insurance, n (%)                   Medicaid       0    41330 (9.6)
                                   Medicare          160560 (37.2)
                                   Other             229341 (53.2)
Hospital length of stay, mean (SD)                0      4.5 (6.6)
In-hospital mortality, n (%)       1              0     8869 (2.1)
One year mortality, n (%)          1              0  106218 (24.6)

### Observable Measurements (Prediction Target)

In [10]:
## NUMERIC FEATURES
## TODO: add mergers across tables and within-tables.

blood_gas = ['so2', 'po2', 'pco2', 'fio2', 'fio2_chartevents', 'aado2', 'aado2_calc', 'pao2fio2ratio', 'ph', 'baseexcess', 'bicarbonate', 'totalco2', 'hematocrit', 'hemoglobin', 'carboxyhemoglobin', 'methemoglobin', 'chloride', 'calcium', 'temperature', 'potassium', 'sodium', 'lactate', 'glucose']

blood_chemistry = ['albumin','globulin','total_protein','aniongap','bicarbonate','bun','calcium','chloride','creatinine','glucose','sodium','potassium']

cardiac_markers = ['troponin_t','ntprobnp','ck_mb']

cbc = ['hematocrit','hemoglobin','mch','mchc','mcv','platelet','rbc','rdw','rdwsd','wbc']

vital_signs = ['heart_rate','sbp','dbp','mbp','sbp_ni','dbp_ni','mbp_ni','resp_rate','temperature','spo2','glucose']

# Glasgow Coma Scale, a measure of neurological function
coma_signs = ['gcs','gcs_motor','gcs_verbal','gcs_eyes','gcs_unable']

renal_out = ['uo_rt_6hr', 'uo_rt_12hr', 'uo_rt_24hr']

renal_creat = ['creat']

renal_aki = ['aki_stage_smoothed']

In [11]:
def filter_measurements(df, hadm_id_selection=None, subject_id_selection=None):
    n = OrderedDict()
    n['n0'] = len(df)
    
    cols = df.columns
    
    # (1) Filter measurements based on admission_id selection.
    if hadm_id_selection is not None:
        df = df[df.hadm_id.isin(hadm_id_selection)]
    
    n['n adm. filter'] = len(df)
    
    
    # Merge with Admission Table
    df_ = df.merge(adm_df, on='hadm_id', how='left')

    # (2) Filter measurements based on subject_id selection
    if subject_id_selection is not None:
        df_ = df_[df_.subject_id.isin(subject_id_selection)]
    
    n['n subj. filter'] = len(df_)
        
    # (3) Filter measurements with time_bin outside the hosp stay.
    mask = df_.time_bin.between(df_.admittime, df_.dischtime)
    
    df_ = df_[mask]
    df = df_[cols]
    n['n in-patient time'] = len(df)
    
    print('\n'.join(f'{filt}: {num} ({100 * num / n["n0"]:.1f})' for filt, num in n.items()))
    print('\n========\n')
    return df

### Numeric Feature Tables

In [12]:
agg_interval = 'hour'
# For higher resolution binning see: 
# https://stackoverflow.com/questions/7299342/what-is-the-fastest-way-to-truncate-timestamps-to-5-minutes-in-postgres

# Time-binning can be specific for each table

In [13]:
renal_out_query = f"""
select icu.hadm_id,
       {', '.join(map(lambda e: f'avg(r.{e}) as {e}', renal_out))},
       date_trunc('{agg_interval}', r.charttime) time_bin
from mimiciv_derived.kdigo_uo as r
inner join mimiciv_icu.icustays as icu
 on icu.stay_id = r.stay_id
group by icu.hadm_id, time_bin
"""

renal_creat_query = f"""
select icu.hadm_id,
       {', '.join(map(lambda e: f'avg(r.{e}) as {e}', renal_creat))},
       date_trunc('{agg_interval}', r.charttime) time_bin
from mimiciv_derived.kdigo_creatinine as r
inner join mimiciv_icu.icustays as icu
 on icu.stay_id = r.stay_id
group by icu.hadm_id, time_bin
"""

renal_aki_query = f"""
select icu.hadm_id,
       {', '.join(map(lambda e: f'avg(r.{e}) as {e}', renal_aki))},
       date_trunc('{agg_interval}', r.charttime) time_bin
from mimiciv_derived.kdigo_stages as r
inner join mimiciv_icu.icustays as icu
 on icu.stay_id = r.stay_id
group by icu.hadm_id, time_bin
"""

renal_out_df = pd.read_sql_query(renal_out_query, con)
renal_creat_df = pd.read_sql_query(renal_creat_query, con)
renal_aki_df = pd.read_sql_query(renal_aki_query, con)


In [14]:
renal_hadm_id = pd.Series(renal_aki_df.hadm_id.unique())

In [15]:
adm_df['adm_has_renal_data'] = adm_df.hadm_id.isin(renal_hadm_id)
adm_df

,subject_id,hadm_id,admittime,dischtime,adm_has_renal_data
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,False
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,False
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,False
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,True
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,False
...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,True
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,False
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,True
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,False


In [16]:
subject_has_renal = adm_df.groupby('subject_id', as_index=False).agg(n_adm=('hadm_id', 'nunique'), adm_has_renal_data=('adm_has_renal_data', 'sum'))
subjects_with_renal_info = subject_has_renal[subject_has_renal['adm_has_renal_data'] > 0]
subjects_with_renal_info['p_adm_with_renal_info'] = subjects_with_renal_info['adm_has_renal_data'] / subjects_with_renal_info['n_adm']
subject_id_with_renal_info = subjects_with_renal_info.subject_id
# subjects_with_renal_info['p_adm_with_renal_info'].hist()

/tmp/ipykernel_404660/2440518339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subjects_with_renal_info['p_adm_with_renal_info'] = subjects_with_renal_info['adm_has_renal_data'] / subjects_with_renal_info['n_adm']


In [17]:
renal_out_df = filter_measurements(renal_out_df, hadm_id_selection=hadm_id_los_geq_12h, 
                                   subject_id_selection=subject_id_with_renal_info)
renal_creat_df = filter_measurements(renal_creat_df, hadm_id_selection=hadm_id_los_geq_12h, 
                                     subject_id_selection=subject_id_with_renal_info)
renal_aki_df = filter_measurements(renal_aki_df, hadm_id_selection=hadm_id_los_geq_12h, 
                                   subject_id_selection=subject_id_with_renal_info)

n0: 3294878 (100.0)
n adm. filter: 3292534 (99.9)
n subj. filter: 3292534 (99.9)
n in-patient time: 3285718 (99.7)


n0: 568158 (100.0)
n adm. filter: 566564 (99.7)
n subj. filter: 566564 (99.7)
n in-patient time: 502249 (88.4)


n0: 3687859 (100.0)
n adm. filter: 3684212 (99.9)
n subj. filter: 3684212 (99.9)
n in-patient time: 3613898 (98.0)




In [18]:
sofa_query = f"""
select hadm_id,
      avg(s.sofa_24hours) as sofa ,
      date_trunc('{agg_interval}', s.endtime) time_bin
from mimiciv_derived.sofa as s
inner join mimiciv_icu.icustays icu on s.stay_id = icu.stay_id
group by hadm_id, time_bin
"""

sofa_df = pd.read_sql_query(sofa_query,con)

In [19]:
sofa_df = filter_measurements(sofa_df, hadm_id_selection=hadm_id_los_geq_12h, 
                            subject_id_selection=subject_id_with_renal_info)

n0: 6044160 (100.0)
n adm. filter: 6037941 (99.9)
n subj. filter: 6037941 (99.9)
n in-patient time: 6001782 (99.3)




In [20]:
blood_gas_query = f"""
select hadm_id,
       {', '.join(map(lambda e: f'avg(bg.{e}) as {e}', blood_gas))},
       date_trunc('{agg_interval}', bg.charttime) time_bin
from mimiciv_derived.bg as bg
group by hadm_id, time_bin
"""

bg_df = pd.read_sql_query(blood_gas_query,con)

In [21]:
bg_df = filter_measurements(bg_df, hadm_id_selection=hadm_id_los_geq_12h, 
                            subject_id_selection=subject_id_with_renal_info)

n0: 491564 (100.0)
n adm. filter: 442007 (89.9)
n subj. filter: 409545 (83.3)
n in-patient time: 404226 (82.2)




In [22]:
blood_chemistry_query = f"""
select hadm_id,
       {', '.join(map(lambda e: f'avg(ch.{e}) as {e}', blood_chemistry))},
       date_trunc('{agg_interval}', ch.charttime) time_bin
from mimiciv_derived.chemistry as ch
group by hadm_id, time_bin
"""

ch_df = pd.read_sql_query(blood_chemistry_query,con)

In [23]:
ch_df = filter_measurements(ch_df, hadm_id_selection=hadm_id_los_geq_12h, 
                            subject_id_selection=subject_id_with_renal_info)

n0: 2530982 (100.0)
n adm. filter: 1987259 (78.5)
n subj. filter: 1334968 (52.7)
n in-patient time: 1313463 (51.9)




In [24]:
cardiac_marker_query = \
f"""
WITH trop AS
(
    SELECT specimen_id, MAX(valuenum) AS troponin_t
    FROM mimiciv_hosp.labevents
    WHERE itemid = 51003
    GROUP BY specimen_id
)
SELECT
    c.hadm_id
    , date_trunc('{agg_interval}', c.charttime) time_bin
    , avg(trop.troponin_t) as troponin_t
    , avg(c.ntprobnp) as ntprobnp
    , avg(c.ck_mb) as ck_mb
FROM mimiciv_hosp.admissions a
LEFT JOIN mimiciv_derived.cardiac_marker c
  ON a.hadm_id = c.hadm_id
LEFT JOIN trop
  ON c.specimen_id = trop.specimen_id
GROUP BY c.hadm_id, time_bin
"""

cardiac_df = pd.read_sql_query(cardiac_marker_query,con)

In [25]:
cardiac_df = filter_measurements(cardiac_df, hadm_id_selection=hadm_id_los_geq_12h, 
                                 subject_id_selection=subject_id_with_renal_info)

n0: 179011 (100.0)
n adm. filter: 177838 (99.3)
n subj. filter: 122907 (68.7)
n in-patient time: 116983 (65.3)




In [26]:
temp_query = \
f"""
WITH temp_t AS (
    SELECT c.hadm_id
        , c.charttime
        , CASE
            WHEN LOWER(c.valueuom)  like '%°f%' THEN (c.valuenum - 32) / 1.8
                ELSE c.valuenum
          END AS temperature
    FROM mimiciv_hosp.admissions a
    INNER JOIN mimiciv_icu.chartevents c
      ON a.hadm_id = c.hadm_id
    INNER JOIN mimiciv_icu.d_items di
      ON c.itemid = di.itemid
    AND c.itemid IN
    (
    227632, -- Arctic Sun/Alsius Temp #1 C
    227634, -- Arctic Sun/Alsius Temp #2 C
    223761 -- Temperature Fahrenheit
    )
    AND valuenum > 10 AND valuenum < 120
)
SELECT temp_t.hadm_id
     , AVG(temp_t.temperature) temperature
     , DATE_TRUNC('{agg_interval}', temp_t.charttime) time_bin
FROM temp_t
GROUP BY temp_t.hadm_id, time_bin
"""
temp_df = pd.read_sql_query(temp_query,con)

In [27]:
temp_df = filter_measurements(temp_df, hadm_id_selection=hadm_id_los_geq_12h, 
                              subject_id_selection=subject_id_with_renal_info)

n0: 1534954 (100.0)
n adm. filter: 1533292 (99.9)
n subj. filter: 1533292 (99.9)
n in-patient time: 1526910 (99.5)




In [28]:
weight_query = f"""
select icu.hadm_id,
     avg(w.weight) weight,
        w.time_bin
 from (
 (select stay_id, w.weight, date_trunc('{agg_interval}', w.starttime) time_bin
  from mimiciv_derived.weight_durations as w)
 union all
 (select stay_id, w.weight, date_trunc('{agg_interval}', w.endtime) time_bin
     from mimiciv_derived.weight_durations as w)
 ) w
inner join mimiciv_icu.icustays icu on w.stay_id = icu.stay_id
group by icu.hadm_id, w.time_bin
"""

weight_df = pd.read_sql_query(weight_query,con)

In [29]:
weight_df = filter_measurements(weight_df, hadm_id_selection=hadm_id_los_geq_12h, 
                                subject_id_selection=subject_id_with_renal_info)

n0: 339479 (100.0)
n adm. filter: 337848 (99.5)
n subj. filter: 337848 (99.5)
n in-patient time: 288349 (84.9)




In [30]:
cbc_query = f"""
select hadm_id,
       {', '.join(map(lambda e: f'avg(cbc.{e}) as {e}', cbc))},
       date_trunc('{agg_interval}', cbc.charttime) time_bin
from mimiciv_derived.complete_blood_count as cbc
group by hadm_id, time_bin
"""
cbc_df = pd.read_sql_query(cbc_query, con)

In [31]:
cbc_df = filter_measurements(cbc_df, hadm_id_selection=hadm_id_los_geq_12h, 
                             subject_id_selection=subject_id_with_renal_info)

n0: 2463587 (100.0)
n adm. filter: 1925075 (78.1)
n subj. filter: 1259401 (51.1)
n in-patient time: 1238087 (50.3)




In [32]:
vital_query = f"""
select icu.hadm_id,
       {', '.join(map(lambda e: f'avg(v.{e}) as {e}', vital_signs))},
       date_trunc('{agg_interval}', v.charttime) time_bin
from mimiciv_derived.vitalsign as v
inner join mimiciv_icu.icustays as icu
 on icu.stay_id = v.stay_id
group by icu.hadm_id, time_bin
"""
vital_df = pd.read_sql_query(vital_query, con)

In [33]:
vital_df = filter_measurements(vital_df, hadm_id_selection=hadm_id_los_geq_12h, 
                               subject_id_selection=subject_id_with_renal_info)

n0: 5886450 (100.0)
n adm. filter: 5880273 (99.9)
n subj. filter: 5880273 (99.9)
n in-patient time: 5861008 (99.6)




In [34]:
gcs_query = f"""
select icu.hadm_id,
       {', '.join(map(lambda e: f'avg(gcs.{e}) as {e}', coma_signs))},
       date_trunc('{agg_interval}', gcs.charttime) time_bin
from mimiciv_derived.gcs as gcs
inner join mimiciv_icu.icustays as icu
 on icu.stay_id = gcs.stay_id
group by icu.hadm_id, time_bin
"""
gcs_df = pd.read_sql_query(gcs_query, con)

In [35]:
gcs_df = filter_measurements(gcs_df, hadm_id_selection=hadm_id_los_geq_12h, 
                             subject_id_selection=subject_id_with_renal_info)

n0: 1628182 (100.0)
n adm. filter: 1626537 (99.9)
n subj. filter: 1626537 (99.9)
n in-patient time: 1621695 (99.6)




In [ ]:

bg_df.to_csv('obs_bg.csv.gz', compression='gzip')
ch_df.to_csv('obs_ch.csv.gz', compression='gzip')
cardiac_df.to_csv('obs_cardiac.csv.gz', compression='gzip')
temp_df.to_csv('obs_temp.csv.gz', compression='gzip')
weight_df.to_csv('obs_weight.csv.gz', compression='gzip')
cbc_df.to_csv('obs_cbc.csv.gz', compression='gzip')
vital_df.to_csv('obs_vital.csv.gz', compression='gzip')
gcs_df.to_csv('obs_gcs.csv.gz', compression='gzip')
renal_out_df.to_csv('obs_renal_out.csv.gz', compression='gzip')
renal_creat_df.to_csv('obs_renal_creat.csv.gz', compression='gzip')
renal_aki_df.to_csv('obs_renal_aki.csv.gz', compression='gzip')
sofa_df.to_csv('obs_sofa.csv.gz', compression='gzip')

### Interventions



In [ ]:
## Inputs - Canonicalise

input_query = \
"""
SELECT
    a.hadm_id
    , inp.starttime as start_time
    , inp.endtime as end_time
    , di.label
    , inp.rate 
    , inp.amount
    , inp.rateuom
    , inp.amountuom
FROM mimiciv_hosp.admissions a
INNER JOIN mimiciv_icu.icustays i
    ON a.hadm_id = i.hadm_id
LEFT JOIN mimiciv_icu.inputevents inp
    ON i.stay_id = inp.stay_id
LEFT JOIN mimiciv_icu.d_items di
    ON inp.itemid = di.itemid
"""


## Procedures - Canonicalise and Refine
icuproc_query = \
"""
SELECT
    a.hadm_id
    , pe.starttime as start_time
    , pe.endtime as end_time
    , di.label
    , pe.value
FROM mimiciv_hosp.admissions a
INNER JOIN mimiciv_icu.icustays i
    ON a.hadm_id = i.hadm_id
LEFT JOIN mimiciv_icu.procedureevents pe
    ON i.stay_id = pe.stay_id
LEFT JOIN mimiciv_icu.d_items di
    ON pe.itemid = di.itemid
"""

hospicdproc_query = \
"""
select pi.hadm_id
, (pi.chartdate)::timestamp as start_time
, (pi.chartdate + interval '1 hour')::timestamp as end_time
, pi.icd_code
, pi.icd_version
, di.long_title
FROM mimiciv_hosp.procedures_icd pi
INNER JOIN mimiciv_hosp.d_icd_procedures di
  ON pi.icd_version = di.icd_version
  AND pi.icd_code = di.icd_code
INNER JOIN mimiciv_hosp.admissions a
  ON pi.hadm_id = a.hadm_id
"""


# === provider order entry (poe) <----- very messy, high irrelevance, ignore

poe_query = f"""
SELECT
    a.hadm_id
    , mimiciv_derived.DATETIME_DIFF(p.ordertime, a.admittime, 'DAY') AS offset
    , p.poe_id
    , p.order_type, p.order_subtype
    , p.transaction_type
    , pd.field_name
    , pd.field_value
FROM mimiciv_hosp.admissions a
INNER JOIN mimiciv_hosp.poe p
    ON a.hadm_id = p.hadm_id
LEFT JOIN  mimiciv_hosp.poe_detail pd
    ON p.poe_id = pd.poe_id
"""

In [ ]:
input_df = pd.read_sql_query(input_query,con)

In [ ]:
icuproc_df = pd.read_sql_query(icuproc_query,con)

In [ ]:
hospicdproc_df = pd.read_sql_query(hospicdproc_query,con)

In [ ]:
# Ignored entirely
# poe_df = pd.read_sql_query(poe_query,con)

In [ ]:
def filter_interventions(df, hadm_id_selection=None, subject_id_selection=None):
    n = OrderedDict()
    n['n0'] = len(df)
    
    cols = df.columns
    
    # (1) Filter interventions based on admission_id selection.
    if hadm_id_selection is not None:
        df = df[df.hadm_id.isin(hadm_id_selection)]
    
    n['n adm. filter'] = len(df)
    
    
    # Merge with Admission Table
    df_ = df.merge(adm_df, on='hadm_id', how='left')

    # (2) Filter interventions based on subject_id selection
    if subject_id_selection is not None:
        df_ = df_[df_.subject_id.isin(subject_id_selection)]
    
    n['n subj. filter'] = len(df_)
        
    # (3) Filter interventions with intervals outside the hosp stay.
    mask1 = df_.start_time.between(df_.admittime, df_.dischtime)
    mask2 = df_.end_time.between(df_.admittime, df_.dischtime)
    n['n start-time filter'] = sum(mask1)
    n['n end-time filter'] = sum(mask1 & mask2)

    df_ = df_[mask1 & mask2]    
    
    df = df_[cols]
    n['n in-patient time'] = len(df)
    
    print('\n'.join(f'{filt}: {num} ({100 * num / n["n0"]:.1f})' for filt, num in n.items()))
    print('\n========\n')
    return df

In [ ]:
input_df = filter_interventions(input_df, hadm_id_selection=hadm_id_los_geq_12h, 
                                subject_id_selection=subject_id_with_renal_info)

In [ ]:
input_df.label.nunique()

In [ ]:
icuproc_df = filter_interventions(icuproc_df, hadm_id_selection=hadm_id_los_geq_12h, 
                                subject_id_selection=subject_id_with_renal_info)

In [ ]:
hospicdproc_df = filter_interventions(hospicdproc_df, hadm_id_selection=hadm_id_los_geq_12h, 
                                subject_id_selection=subject_id_with_renal_info)

In [ ]:
hospicdproc_df

### `hospicdproc` + `icuproc` Further filteration and grouper

In [ ]:
df = hospicdproc_df.merge(adm_df, on='hadm_id', how='left')

# timeperc_within_stay = (df['start_time'] - df['admittime']).dt.total_seconds() /(df['dischtime'] - df['admittime']).dt.total_seconds()
# timeperc_within_stay.plot.kde()

In [ ]:
icd_n_subjects = df.groupby(['icd_code', 'long_title', 'icd_version'], as_index=False).agg(n_subjects=('subject_id', 'nunique'))
icd_n_subjects['p_subjects'] = icd_n_subjects['n_subjects'] / len(subject_id_with_renal_info)
# icd_n_subjects['p_subjects'].plot.kde()

In [ ]:
# Conisder ICD codes with minimum coverage of 0.5% of the selected subjects.
hospicd_R1 = icd_n_subjects[icd_n_subjects['p_subjects'] > 0.005]

# Remove procedures that are purely diagnostic.
patterns = ['diag', 'fluoro', 'biops', 'inspection', 'bronchoscop', 'monitor', 'ultrasonography']

hospicd_R2_mask = hospicd_R1['long_title'].str.match('|'.join(f'(.*{p}.*)' for p in patterns), case=False)
hospicd_R2 = hospicd_R1[~hospicd_R2_mask]


hospicd_R1.to_csv('hospicd_R1.csv')
hospicd_R2.to_csv('hospicd_R2.csv')

In [ ]:
# Apply filteration.
hospicdproc_df = hospicdproc_df[hospicdproc_df.icd_code.isin(hospicd_R2.icd_code)]

In [ ]:
df = hospicdproc_df.merge(adm_df, on='hadm_id', how='left')

# timeperc_within_stay = (df['start_time'] - df['admittime']).dt.total_seconds() /(df['dischtime'] - df['admittime']).dt.total_seconds()
# timeperc_within_stay.plot.kde(bw_method=0.05)

In [ ]:
icd_n_subjects = df.groupby(['icd_code', 'long_title', 'icd_version'], as_index=False).agg(n_subjects=('subject_id', 'nunique'))
icd_n_subjects['p_subjects'] = icd_n_subjects['n_subjects'] / len(subject_id_with_renal_info)
# icd_n_subjects['p_subjects'].plot.kde(bw_method=0.01)

In [ ]:
hospicdproc_df

In [ ]:
hospicdproc_grouper = pd.read_csv('int_grouper_hospicdproc.csv')
hospicdproc_grouper

In [ ]:
n1 = len(hospicdproc_df)
icd_selection = set(hospicdproc_grouper.icd_code)
print(hospicdproc_grouper.icd_code.nunique(), len(hospicdproc_grouper))



In [ ]:
hospicdproc_df = hospicdproc_df[hospicdproc_df.icd_code.isin(icd_selection)]
n2 = len(hospicdproc_df)
n1, n2

#### Standard Procedure Table

In [ ]:
hospicd_coder = lambda v, c: f"v{v}:c{c}"

hospicdproc_df["code"] = list(map(hospicd_coder, hospicdproc_df["icd_version"], hospicdproc_df["icd_code"]))
hospicdproc_grouper["code"] = list(map(hospicd_coder, hospicdproc_grouper["icd_version"], hospicdproc_grouper["icd_code"]))
hospicdproc_grouper["label"] = hospicdproc_grouper["long_title"]

hospicdproc_df = hospicdproc_df[["hadm_id", "start_time", "end_time", "code"]]
hospicdproc_grouper = hospicdproc_grouper[["code", "group", "label"]]

### 'icuproc` refinement

In [ ]:
icuproc_df

In [ ]:
icuproc_df['total_interval_hrs'] = (icuproc_df['end_time'] - icuproc_df['start_time']).dt.total_seconds() / 3600
icuproc_df['total_interval_mins'] = (icuproc_df['end_time'] - icuproc_df['start_time']).dt.total_seconds() / 60
icuproc_df['value_per_hour'] = icuproc_df['value'] / icuproc_df['total_interval_hrs']

df = icuproc_df.merge(adm_df, on='hadm_id', how='left')
df['count'] = 1

In [ ]:
icuproc_n_subjects = df.groupby('label', as_index=False).agg(
    n_subjects=('subject_id', 'nunique'),
    n=('count', 'sum'),
    mean_value=('value', 'mean'),
    std_value=('value', 'std'),
    mean_value_per_hour=('value_per_hour', 'mean'),
    std_value_per_hour=('value_per_hour', 'std'),
    mean_interval_minutes=('total_interval_mins', 'mean'),
    std_interval_minutes=('total_interval_mins', 'std'))
    

In [ ]:
icuproc_n_subjects.to_csv('icuproc_n_subjects.csv')

In [ ]:
int_icuproc_grouper = pd.read_csv('int_grouper_icuproc.csv')

In [ ]:
n1 = len(icuproc_df)
exclude_icuproc = set(int_icuproc_grouper[int_icuproc_grouper.group == 'exclude'].label)

icuproc_df = icuproc_df[~icuproc_df.label.isin(exclude_icuproc)]
n2 = len(icuproc_df)
n1, n2

#### Standard Procedure Table

In [ ]:
icuproc_df["code"] = icuproc_df["label"]
int_icuproc_grouper["code"] = int_icuproc_grouper["label"]
int_icuproc_grouper["label"] = int_icuproc_grouper["label"]

icuproc_df = icuproc_df[["hadm_id", "start_time", "end_time", "code"]]
int_icuproc_grouper = int_icuproc_grouper[["code", "group", "label"]]

In [ ]:
int_proc_df = pd.concat([hospicdproc_df, icuproc_df], axis=0)
int_grouper_proc = pd.concat([hospicdproc_grouper, int_icuproc_grouper], axis=0)

int_proc_df.to_csv('int_proc.csv.gz', compression='gzip')
int_grouper_proc.to_csv('int_grouper_proc.csv.gz', compression='gzip')

### `input` filteration and refinement

In [ ]:
input_df['total_interval_hrs'] = (input_df['end_time'] - input_df['start_time']).dt.total_seconds() / 3600
input_df['total_interval_mins'] = (input_df['end_time'] - input_df['start_time']).dt.total_seconds() / 60
input_df['amount_per_hour'] = input_df['amount'] / input_df['total_interval_hrs']

In [ ]:
df = input_df.merge(adm_df, on='hadm_id', how='left')
df['count'] = 1

In [ ]:
input_n_subjects = df.groupby(['label', 'amountuom'], as_index=False).agg(
    n_subjects=('subject_id', 'nunique'),
    n=('count', 'sum'),
    mean_amount=('amount', 'mean'),
    std_amount=('amount', 'std'),
    mean_amount_per_hour=('amount_per_hour', 'mean'),
    std_amount_per_hour=('amount_per_hour', 'std'),
    mean_interval_minutes=('total_interval_mins', 'mean'),
    std_interval_minutes=('total_interval_mins', 'std'))
    

In [ ]:
input_n_subjects.to_csv('input_n_subjects.csv')


In [ ]:
"""
from IPython.display import display

input_n_subjects_tom = pd.read_csv('input_n_subjects_tom.csv', index_col=[0])
# Change 'M' (maybe) to 'Y' (yes)
input_n_subjects_tom.loc[input_n_subjects_tom['dose_impact'] == 'M', 'dose_impact'] = 'Y'

# Add column for unit normalisation, default=1.0
input_n_subjects_tom['rate_normaliser'] = 1.0

# Add label groups
input_n_subjects_tom['group'] = input_n_subjects_tom.label.map(input_group_map)

# Default group decision:

init_decision = {}
for group, group_df in input_n_subjects_tom.groupby('group'):
    dose_impact = list(group_df['dose_impact'].unique())
    
    assert len(dose_impact) == 1, f"Incosistant dose impact decision {group}."
    dose_impact = dose_impact[0]
    if dose_impact == 'Y':
        if group_df.label.nunique() > 1:
            if group_df.amountuom.nunique() > 1:
                init_decision[group] = 'DS(or DH, norm)'
            else:
                init_decision[group] = 'DS(or DH)'
        else:
            if group_df.amountuom.nunique() > 1:
                init_decision[group] = 'DH(norm)'
            else:
                init_decision[group] = 'DH'
    elif dose_impact == 'N':
        
        if group_df.label.nunique() > 1:
            init_decision[group] = 'H(or S)'
        else:
            init_decision[group] = 'H'
    else:
        init_decision[group] = 'E'

input_n_subjects_tom['group_decision'] = input_n_subjects_tom['group'].map(init_decision)

# Change the order of rate_normaliser column
input_n_subjects_tom.insert(1, 'rate_normaliser', input_n_subjects_tom.pop('rate_normaliser'))

input_n_subjects_tom = input_n_subjects_tom.set_index(['group', 'dose_impact', 'group_decision', 'label', 'amountuom'])
"""

In [ ]:
"""
with pd.ExcelWriter('input_label_groups_notes.xlsx') as writer:
    input_n_subjects_tom.to_excel(writer, sheet_name='Sheet1', merge_cells = True)
"""

In [ ]:
int_grouper_input_df = pd.read_excel('input_label_groups_decisions.xlsx').fillna(method='ffill')

In [ ]:
int_grouper_input_df

In [ ]:
int_grouper_input_df.to_csv('int_grouper_input.csv.gz', compression='gzip')

In [ ]:
input_df

In [ ]:
input_df.to_csv('int_input.csv.gz', compression='gzip')

## Dx Codes Filteration

In [ ]:
def filter_discharge_codes(df, subject_id_selection=None):
    n = OrderedDict()
    n['n0'] = len(df)
    
    cols = df.columns
    
    # (1) Filter discharge codes if subject has a single admission.
    n_adms = df.groupby('subject_id', as_index=False).agg(n_adms=('hadm_id', 'nunique'))
    n_adms_subjects = set(n_adms[n_adms['n_adms'] > 1].subject_id)
    df = df[df.subject_id.isin(n_adms_subjects)]
    
    n['n adm. > 1 filter'] = len(df)
    

    # (2) Filter discharge dx codes based on subject_id selection
    if subject_id_selection is not None:
        df = df[df.subject_id.isin(subject_id_selection)]
    
    n['n subj. filter'] = len(df)
        
    
    print('\n'.join(f'{filt}: {num} ({100 * num / n["n0"]:.1f})' for filt, num in n.items()))
    print('\n========\n')
    return df

In [ ]:
dx_query = f"""
SELECT
    hadm_id, subject_id, icd_code, icd_version
FROM mimiciv_hosp.diagnoses_icd 
"""
dx_df = pd.read_sql_query(dx_query,con)

In [ ]:
dx_df = filter_discharge_codes(dx_df, subject_id_selection=subject_id_with_renal_info)

In [ ]:
dx_df

In [ ]:
dx_df.to_csv('dx_df.csv.gz', compression='gzip')

## Admissions & Static Tables 

In [ ]:
def filter_admissions(df, subject_id_selection=None):
    n = OrderedDict()
    n['n0'] = len(df)
    
    cols = df.columns

    # (1) Filter discharge dx codes based on subject_id selection
    if subject_id_selection is not None:
        df = df[df.subject_id.isin(subject_id_selection)]
    
    n['n subj. filter'] = len(df)
        
    
    print('\n'.join(f'{filt}: {num} ({100 * num / n["n0"]:.1f})' for filt, num in n.items()))
    print('\n========\n')
    return df

In [ ]:
adm_query = f"""
SELECT
    hadm_id, subject_id, admittime, dischtime, race
FROM mimiciv_hosp.admissions 
"""
adm_df = pd.read_sql_query(adm_query,con)

In [ ]:
static_query = f"""
SELECT 
    p.subject_id, p.gender, a.race, p.anchor_age, p.anchor_year
    from mimiciv_hosp.patients p
    left join 
    (select subject_id, max(race) as race
    from mimiciv_hosp.admissions
    group by subject_id) as a
    on p.subject_id = a.subject_id
"""
static_df = pd.read_sql_query(static_query,con)

In [ ]:
adm_df = filter_admissions(adm_df, subject_id_selection=subject_id_with_renal_info)
static_df = filter_admissions(static_df, subject_id_selection=subject_id_with_renal_info)


In [ ]:
adm_df.to_csv('adm_df.csv.gz', compression='gzip')
static_df.to_csv('static_df.csv.gz', compression='gzip')